In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [2]:
df = pd.read_csv("hindi-train.csv")

In [3]:
def count_length():
    df['word_count'] = df['headline'].apply(lambda x: len(str(x).split(" ")))

In [4]:
count_length()
df.tail(10)

,headline,Label,word_count
37490,'सूर्यवंशी' और 'बेल बॉटम' की रिलीज पर अक्षय कु...,Entertainment,15
37491,"श्रेया घोषाल ने दी फैन्स को गुड न्यूज, बेबी ब्...",Entertainment,13
37492,EXCLUSIVE: पहले ऑडिशन के बाद एक हफ्ते तक घर से...,Entertainment,22
37493,मेहंदी फोटो शेयर कर अली फजल-ऋचा चड्ढा ने दी अप...,Entertainment,22
37494,'राम सेतु' की शूटिंग इस महीने से फिर शुरू करें...,Entertainment,19
37495,"प्लेन में बच्चे का जन्म, लड़की ने टॉयलेट पेपर म...",Entertainment,15
37496,अंतिम यात्रा में भी पार्टनर का साथ नहीं छोड़ रह...,Entertainment,14
37497,"बॉयफ्रेंड से मिलने रूस से PAK पहुंच गई लड़की, फ...",Entertainment,16
37498,"वेडिंग फोटोशूट करा रहे थे दूल्हा-दुल्हन, कीचड़ ...",Entertainment,13
37499,बिना मास्क के पहुंची महिला को दुकानदार ने टोंक...,Entertainment,15


In [5]:
import re
def processText(text):
    text = text.lower()
    text = re.sub("[a-zA-z0-9]",' ',text)
    text = re.sub("((www.[^s]+)|(https?://[^s]+))",'',text)
    text = re.sub("@[^s]+",'',text)
    text = re.sub("[s]+", ' ', text)
    text = re.sub(r"#([^s]+)", r'', text)
    text = text.strip('"')
    return text

In [6]:
for i in range(len(df)):
    df['headline'][i] = processText(df['headline'][i])

In [7]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens
df['headline'] = df['headline'].apply(lambda x: tokenization(x))

In [8]:
for i in range(len(df)):
    df['headline'][i] = [s.replace("\n", "") for s in df['headline'][i]]

In [9]:
stopwords_hi = ['तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','आगे','माननीय','शहर','बताएं','कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें','आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर', 'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'तो', 'था', 'थी', 'थे', 'दबारा', 'दिया', 'दुसरा', 'दूसरे', 'दो', 'द्वारा', 'न', 'नहीं', 'ना', 'निहायत', 'नीचे', 'ने', 'पर', 'पर', 'पहले', 'पूरा', 'पे', 'फिर', 'बनी', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल', 'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे', 'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वर्ग', 'वह', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'वग़ैरह', 'संग', 'सकता', 'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'ही', 'हुआ', 'हुई', 'हुए', 'है', 'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने', 'अपनि', 'जेसे', 'होति', 'सभि', 'तिंहों', 'इंहों', 'दवारा', 'इसि', 'किंहें', 'थि', 'उंहों', 'ओर', 'जिंहें', 'वहिं', 'अभि', 'बनि', 'हि', 'उंहिं', 'उंहें', 'हें', 'वगेरह', 'एसे', 'रवासा', 'कोन', 'निचे', 'काफि', 'उसि', 'पुरा', 'भितर', 'हे', 'बहि', 'वहां', 'कोइ', 'यहां', 'जिंहों', 'तिंहें', 'किसि', 'कइ', 'यहि', 'इंहिं', 'जिधर', 'इंहें', 'अदि', 'इतयादि', 'हुइ', 'कोनसा', 'इसकि', 'दुसरे', 'जहां', 'अप', 'किंहों', 'उनकि', 'भि', 'वरग', 'हुअ', 'जेसा', 'नहिं']
stopwords_en = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
punctuations = [".",",","?","/","<",">","!",":",";","'","[","]","{","}","(",")","-","@","#","$","%","^","&","*"]
to_be_removed = stopwords_hi + punctuations + stopwords_en

In [10]:
for i in range(len(df)):
    df['headline'][i]=[ele for ele in df['headline'][i] if ele not in (to_be_removed)]
count_length()
df.tail(10)

,headline,Label,word_count
37490,"[सूर्यवंशी, बेल, बॉटम, रिलीज, अक्षय, कुमार, बय...",Entertainment,10
37491,"[श्रेया, घोषाल, दी, फैन्स, गुड, न्यूज, बेबी, ब...",Entertainment,9
37492,"[ऑडिशन, हफ्ते, बाहर, निकले, ताहिर, राज, भसीन, ...",Entertainment,10
37493,"[मेहंदी, फोटो, शेयर, अली, फजल, ऋचा, चड्ढा, दी,...",Entertainment,17
37494,"[राम, सेतु, शूटिंग, महीने, शुरू, करेंगे, अक्षय...",Entertainment,13
37495,"[प्लेन, बच्चे, जन्म, लड़की, टॉयलेट, पेपर, लपेट,...",Entertainment,9
37496,"[अंतिम, यात्रा, पार्टनर, छोड़, मोर, देखें, इमोश...",Entertainment,8
37497,"[बॉयफ्रेंड, मिलने, रूस, पहुंच, गई, लड़की, अचानक...",Entertainment,10
37498,"[वेडिंग, फोटोशूट, करा, दूल्हा, दुल्हन, कीचड़, ध...",Entertainment,8
37499,"[बिना, मास्क, पहुंची, महिला, दुकानदार, टोंका, ...",Entertainment,10


In [11]:
import collections
from collections import Counter
corpus_list =[]
for i in range(len(df)):
    corpus_list +=df['headline'][i]
counter=collections.Counter(corpus_list)

In [12]:
least_common= [word for word, word_count in Counter(corpus_list).most_common()[:-50:-1]]
for i in range(len(df)):
    df['headline'][i]=[ele for ele in df['headline'][i] if ele not in (least_common)]

In [13]:
df

,headline,Label,word_count
0,"[मेलबर्न, एयरपोर्ट, घंटों, फंसे, नोवाक, जोकोवि...",Sports,12
1,"[अर्जेंटीना, स्टार, फुटबॉलर, लियोनल, मेसी, कोव...",Sports,11
2,"[बड़ा, फैसला, कैंपों, प्रैक्टिस, सेंटर, पहुंचन...",Sports,12
3,"[ऑलराउंडर, रोहित, गूलिया, दम, हरियाणा, स्टीलर्...",Sports,10
4,"[टेनिस, टाटा, ओपन, महाराष्ट्र, भारत, युकी, भां...",Sports,9
...,...,...,...
37495,"[प्लेन, बच्चे, जन्म, लड़की, टॉयलेट, पेपर, लपेट,...",Entertainment,9
37496,"[अंतिम, यात्रा, पार्टनर, छोड़, मोर, देखें, इमोश...",Entertainment,8
37497,"[बॉयफ्रेंड, मिलने, रूस, पहुंच, गई, लड़की, अचानक...",Entertainment,10
37498,"[वेडिंग, फोटोशूट, करा, दूल्हा, दुल्हन, कीचड़, ध...",Entertainment,8


In [14]:
df['headline'] = df['headline'].str.join(" ")
print(df.tail(10))

                                                headline          Label  \
37490  सूर्यवंशी बेल बॉटम रिलीज अक्षय कुमार बयान फैन्...  Entertainment   
37491    श्रेया घोषाल दी फैन्स गुड न्यूज बेबी ब्वॉय जन्म  Entertainment   
37492  ऑडिशन हफ्ते बाहर निकले ताहिर राज भसीन बदली किस्मत  Entertainment   
37493  मेहंदी फोटो शेयर अली फजल ऋचा चड्ढा दी शादी अफव...  Entertainment   
37494  राम सेतु शूटिंग महीने शुरू करेंगे अक्षय कुमार ...  Entertainment   
37495  प्लेन बच्चे जन्म लड़की टॉयलेट पेपर लपेट डस्टबिन...  Entertainment   
37496   अंतिम यात्रा पार्टनर छोड़ मोर देखें इमोशनल वीडियो  Entertainment   
37497  बॉयफ्रेंड मिलने रूस पहुंच गई लड़की अचानक होटल ख...  Entertainment   
37498    वेडिंग फोटोशूट करा दूल्हा दुल्हन कीचड़ धड़ाम गिरे  Entertainment   
37499  बिना मास्क पहुंची महिला दुकानदार टोंका उसने कप...  Entertainment   

       word_count  
37490          10  
37491           9  
37492          10  
37493          17  
37494          13  
37495           9  
37496           8  
37497         

In [15]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['headline'],df['Label'],test_size=0.3, random_state=42)

In [16]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
print(Train_Y[0:10])
print(Train_X[0:10])

[1 2 1 2 2 3 4 4 2 3]
36768    किताबों ढेर टेलीस्कोप मेटल आर्टवर्क मुंबई खूबस...
23832                  कांग्रेस पलटवार बंद करो ओछी राजनीति
34090     अंतिम यात्रा पार्टनर छोड़ मोर देखें इमोशनल वीडियो
24751    ‘ अटकी शिवसेना कैसा सबका सबका विकास ’ असदुद्दी...
28923    जीतेगा गुजरात चुनाव मतदान तारीखों घोषणा शुरू च...
6460     क्रिस्टियानो रोनाल्डो गर्लफ्रेंड जॉर्जिना बेटे...
12623                 नए वायरलेस इयरफोन्स लॉन्च कीमत रुपये
10325    फैन्स खुशखबरी जल्द भारत लॉन्च होगा नया गेम अगर...
24197             बीजेपी नेता बताया मेरा फोन टैप संजय राउत
3426     हरियाणा बबीता फोगाट कविता देवी खेल विभाग उप नि...
Name: headline, dtype: object


In [17]:
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(df['headline'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [18]:
dt = RandomForestClassifier(max_depth = 250 ,n_estimators = 450)
dt.fit(Train_X_Tfidf,Train_Y)
predictions_dec = dt.predict(Test_X_Tfidf)

In [19]:
print("RFC Accuracy Score -> ",accuracy_score(predictions_dec, Test_Y)*100)

RFC Accuracy Score ->  74.59555555555556


In [20]:
import joblib
import os
loaded_model = joblib.dump(dt, 'hin_rfc.pkl')

In [21]:
mat = confusion_matrix(predictions_dec, Test_Y)
mat

array([[1784,   82,   86,   65,  123],
       [  48, 1539,   91,  195,  342],
       [ 182,  191, 1880,  244,   66],
       [  54,   82,  107, 1492,   31],
       [ 118,  411,   42,  298, 1697]], dtype=int64)

In [22]:
report = classification_report(predictions_dec, Test_Y)
print(report)

              precision    recall  f1-score   support

           0       0.82      0.83      0.82      2140
           1       0.67      0.69      0.68      2215
           2       0.85      0.73      0.79      2563
           3       0.65      0.84      0.73      1766
           4       0.75      0.66      0.70      2566

    accuracy                           0.75     11250
   macro avg       0.75      0.75      0.75     11250
weighted avg       0.75      0.75      0.75     11250



In [23]:
#business
simple_test = ["एसबीआई ने ग्राहकों को किया हाई अलर्ट! बंद हो सकती है आपकी बैंकिंग सेवा, जानिए वजह"]
Train_X_Tfidf = Tfidf_vect.transform(simple_test)
pred1 = dt.predict(Train_X_Tfidf)
print(pred1[0])
predictions_test = Encoder.inverse_transform(pred1)
predictions_test[0]

0


'Business'

In [24]:
#entertainment
simple_test = ["शादी की खबरों के बीच कटरीना कैफ पर भड़के विक्की कौशल, दोनों के बीच हुआ झगड़ा! जानें वजह"]
Train_X_Tfidf = Tfidf_vect.transform(simple_test)
pred2 = dt.predict(Train_X_Tfidf)
print(pred2[0])
predictions_test = Encoder.inverse_transform(pred2)
predictions_test[0]

1


'Entertainment'

In [25]:
#political
simple_test = ["पश्चिमी यूपी से पूर्वांचल, अवध से बुंदेलखंड तक, कैसे कमल की फसल को खाद दे रहे पीएम?"]
Train_X_Tfidf = Tfidf_vect.transform(simple_test)
pred3 = dt.predict(Train_X_Tfidf)
print(pred3[0])
predictions_test = Encoder.inverse_transform(pred3)
predictions_test[0]

2


'Politics'

In [26]:
#sports
simple_test = ["भारत पहुंची पाकिस्तान की टीम, 24 नवंबर से भुवनेश्वर में खेला जाएगा वर्ल्ड कपाा"]
Train_X_Tfidf = Tfidf_vect.transform(simple_test)
pred4 = dt.predict(Train_X_Tfidf)
print(pred4[0])
predictions_test = Encoder.inverse_transform(pred4)
predictions_test[0]

3


'Sports'

In [27]:
#tech
simple_test = ["सावधान! अगर ये 7 ऐप्स आपके फोन में हों तो तुरंत करें डिलीट, बन सकते हैं खतरा"]
Train_X_Tfidf = Tfidf_vect.transform(simple_test)
pred5 = dt.predict(Train_X_Tfidf)
print(pred5[0])
predictions_test = Encoder.inverse_transform(pred5)
predictions_test[0]

4


'Tech'